# OpenAI Embedding API로 문서 임베딩하고 검색하기

In [1]:
!pip install tiktoken

In [1]:
import math
import os
from glob import glob
import tiktoken

## Tiktoken?

Tiktoken은 OpenAI에서 측정는 텍스트의 토큰 갯수를 확인 할 수 있는 방법입니다.

In [2]:
encoding = tiktoken.get_encoding("cl100k_base")
num_tokens = len(encoding.encode("tiktoken is great!"))

In [3]:
num_tokens

6

In [4]:
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [5]:
encoding.decode(encoding.encode("tiktoken is great!"))

'tiktoken is great!'

## 긴 문서 chunking 하기

In [6]:
def long_text_into_chunks(text, n_slide=400, max_size=800): # n_slide=4000, max_size=8000, MAX_TOKENS = 8191
    text_chunk_list = []

    encoding = tiktoken.get_encoding("cl100k_base")
    token_ids = encoding.encode(text)

    n_chunk = int(math.ceil(len(token_ids) / n_slide))

    for chunk_i in range(n_chunk):
        token_ids_chunk = token_ids[chunk_i * n_slide:chunk_i * n_slide + max_size]
        curr_text_chunk = encoding.decode(token_ids_chunk)
        text_chunk_list.append(curr_text_chunk)
    return text_chunk_list

In [7]:
# get all rst files
def get_all_doc_data(dir_path, doc_id_start=0):
    md_path_list = glob(f"{dir_path}/**/*.rst", recursive=True)
    doc_list = []
    # make docs
    for doc_id, md_path in enumerate(md_path_list, start=doc_id_start):
        with open(md_path, "rt") as f:
            text = f.read()
        doc_list.append({"docId": str(doc_id), "src": md_path, "text": text})

    psg_list = []

    for doc in doc_list:
        text_chunk_list = long_text_into_chunks(doc['text'])
        for psg_id, text_chunk in enumerate(text_chunk_list):

            psg_list.append(text_chunk)

    return psg_list

In [8]:
dir_path = "./konlpy"
psg_list = get_all_doc_data(dir_path, doc_id_start=0)

In [9]:
len(psg_list)

56

In [10]:
print(psg_list[2])

 # Install `Apache Ant <http://ant.apache.org/manual/install.html>`_
        make java

    1. 코드를 단 한 줄이라도 수정한 모든 경우::

        pip install -r requirements-dev.txt
        pip3 install -r requirements-dev.txt
        make build      # create tar.gz
        make check      # check code styles
        make testall    # run tests

- PR을 보내기 전 다음을 확인해주세요.
    1. PR을 보내면 해당 코드는 KoNLPy의 오픈소스 라이센스를 따름
    1. PR를 보낸 후 코드의 일부를 변경하도록 요청될 경우, ``git commit --amend`` 로 커밋을 수정


4. 문서 수정하기
----------------

- 오류 수정: 사소한 오타 등을 발견하여 수정을 요청하고 싶은 경우, 문서를 직접 수정하실 수 있습니다.
- 내용 추가: 그 외에도 내용이 잘 이해되지 않는 부분이 있다거나, 설명이 부족한 부분에도 내용을 추가하실 수 있습니다. 특히, KoNLPy는 예제가 풍부한 문서를 지향합니다. 문서에 추가하면 좋을 법한 예제가 있으면 공유해주세요.
- 번역: KoNLPy의 문서 영문과 한국어를 지원합니다. 표현이 어색하거나 번역이 덜 된 부분을 수정하실 수 있습니다.

> Note: 기왕이면 기여하신 부분에 대해 정확한 attribution을 할 수 있도록, 다음의 과정대로 문서를 설치하고 수정한 후, PR을 보내주시기 바랍니다. 만일 이 과정이 너무 어렵거나 번거롭고, attribution은 따로 받지 않아도 된다면, konlpy@googlegroups.com로 메일을 보내주셔도 됩니다.


Setup docs
''''''''''

1. Fork and clone KoNLPy::

    

## 문서 Embedding 하기

In [11]:
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI
client = OpenAI()

In [12]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def encode(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002")
    return np.array(response.data[0].embedding, dtype=np.float32)

In [13]:
psg_embs = np.array([encode(psg) for psg in psg_list], dtype=np.float32)

In [14]:
psg_embs

array([[ 0.00967265, -0.00666077,  0.01922067, ...,  0.00560141,
        -0.0002207 , -0.03148977],
       [ 0.00923838, -0.01505167,  0.01927951, ...,  0.01457001,
        -0.00302371, -0.01349298],
       [ 0.01724748, -0.0034495 ,  0.0238425 , ...,  0.01991721,
        -0.0007711 , -0.00141168],
       ...,
       [-0.01179341, -0.00282618,  0.01642885, ..., -0.00837972,
        -0.00548347, -0.04036063],
       [-0.02807183,  0.00458171,  0.01739063, ..., -0.00424482,
         0.00233873, -0.0209794 ],
       [-0.00574583,  0.01449099,  0.02416089, ...,  0.00822911,
        -0.02133473, -0.03003487]], dtype=float32)

In [15]:
psg_embs.shape

(56, 1536)

## Faiss Index

### Faiss Index 빌드하기

In [21]:
!pip install faiss-cpu

In [17]:
import faiss

In [18]:
# Calculate the L2 norms for each vector (row)
norms = np.linalg.norm(psg_embs, axis=1, keepdims=True)

# Divide by the norm to normalize
psg_embs_normed = psg_embs / norms

In [19]:
psg_embs_normed.shape

(56, 1536)

In [20]:
index = faiss.IndexFlatIP(psg_embs_normed.shape[1])  # 1536

In [21]:
index.add(psg_embs_normed)

### Faiss Index에서 검색하기

In [22]:
query = "Standing on the shoulders of giants"  # Ubuntu install, Standing on the shoulders of giants
query_emb = encode(query)
query_emb_normed =  query_emb / np.linalg.norm(query_emb)

In [23]:
query_emb_normed

array([-0.00670303, -0.03380325, -0.00342194, ..., -0.00606922,
       -0.01421273, -0.00804748], dtype=float32)

In [24]:
dist_list_list, psg_idx_list_list = index.search(query_emb.reshape(1, -1), k=3)  # (1, dim 1536)

In [25]:
dist_list_list

array([[0.75514424, 0.7136526 , 0.70839906]], dtype=float32)

In [26]:
psg_idx_list_list

array([[13, 36, 49]])

In [27]:
dist_list, psg_idx_list = dist_list_list[0], psg_idx_list_list[0]

In [28]:
for psg_idx in psg_idx_list:
    print(psg_list[psg_idx])
    print("*"*30)

고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))
    [(오류, NNG),
     (보고, NNG),
     (는, JX),
     (실행, NNG),
     (환경, NNG),
     (,, SP),
     (에러, NNG),
     (메세지, NNG),
     (와, JKM),
     (함께, MAG),
     (설명, NNG),
     (을, JKO),
     (최대한, NNG),
     (상세히, MAG),
     (!, SF),
     (^^, EMO)]


Standing on the shoulders of giants
-----------------------------------

Korean, the `13th most widely spoken language in the world <http://www.koreatimes.co.kr/www/news/nation/2014/05/116_157214.html>`_, is a beautiful, yet complex language.
Myriad :ref:`engines` were built by numerous researchers, to computationally extract meaningful features from the labyrinthine text.

KoNLPy is not just to create another, but to unify and build upon their shoulders, and see one step further.
It is built particularly in the `Python (programming) language <http://python.org>`_, not only because of the language's simplicity and elegance, but also the powerful string processing modules and applicability to various tas

In [29]:
psg_idx_list_list

array([[13, 36, 49]])